In [1]:
import pandas as pd
import altair as alt

In [2]:
data = pd.read_excel('../data/2014-2020.xlsx')
data.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN


In [3]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [4]:
data.drop(['Unnamed: 12', 'UK_MLD',  'UK_EURO', 'UK_BLR_RUS', 'GAES_PUMP'], axis = 1, inplace = True)
data.columns = ['t/d', 'aes', 'tec', 'vde', 'tes', 'ges', 'geas', 'consumption']

data[['hour', 'date']] = data['t/d'].str.split('-', expand=True)
data.drop(['t/d'], axis = 1, inplace = True)

In [5]:
data[['day', 'month', 'year']] = data['date'].str.split('.', expand=True)
data.drop(['date'], axis = 1, inplace = True)

data[['hour','day', 'month', 'year']] = data[['hour','day', 'month', 'year']].astype(int)
data.head()

,aes,tec,vde,tes,ges,geas,consumption,hour,day,month,year
0,9235,2039,621,4942,385.0,0,16693,24,31,12,2020
1,9221,2159,707,5549,470.0,0,17805,23,31,12,2020
2,9249,2377,709,5906,1000.0,0,18870,22,31,12,2020
3,9256,2499,702,6329,909.0,322,19887,21,31,12,2020
4,9213,2521,702,6640,823.0,602,20387,20,31,12,2020


### Як змінювалась структура генерації електроенергії за роками?

In [6]:
data1 = data.copy(deep=False)
data1.drop(['consumption'], axis = 1, inplace = True)
data1 = data1.groupby('year')[['aes', 'tec', 'vde', 'tes', 'ges', 'geas']].sum().reset_index()
data1 = pd.melt(data1, id_vars=['year'], var_name='stations', value_name='values')
data1['stations'] = data1['stations'].astype(str)

In [7]:
alt.Chart(data1).mark_bar().encode(
    x = alt.X('year:N'),
    y = alt.Y('values:Q'),
    color = alt.Color('stations:N'),
    tooltip = alt.Tooltip('values')
).properties(width = 650, height = 500, background = '#F9F9F9', padding = 25)

alt.Chart(...)

### Як залежить споживання електроенергії від дня року та години доби?

In [8]:
data2 = data.copy(deep=False)
data2.drop(['aes', 'tec', 'vde', 'tes', 'ges', 'geas'], axis = 1, inplace = True)
data2['day_of_the_year'] = data2['day'].astype(str) + "." + data2['month'].astype(str)
data2 = data2.groupby(['day_of_the_year', 'hour'])[['consumption','day', 'month']].mean().reset_index()
data2[['day', 'month']] = data2[['day', 'month']].astype(int)
data2 = data2.sort_values(by = ['month', 'day', 'hour']).reset_index()

In [9]:
alt.Chart(data2).mark_rect().encode(
    x = alt.X('hour:N'),
    y = alt.Y('day_of_the_year:N', sort = alt.Sort(field = 'month', op = 'mean', order = 'ascending')),
    color = alt.Color('consumption:Q', scale=alt.Scale(scheme='purplegreen')),
    tooltip = alt.Tooltip('consumption')
).properties(width = 650, background = '#F9F9F9', padding = 25)

alt.Chart(...)

### Як змінюється генерація електроенергії з різних джерел впродовж доби?

In [10]:
data3 = data.copy(deep=False)
data3.drop(['consumption'], axis = 1, inplace = True)
data3 = data3.groupby('hour')[['aes', 'tec', 'vde', 'tes', 'ges', 'geas']].mean().reset_index()
data3 = pd.melt(data3, id_vars=['hour'], var_name='stations', value_name='values')
data3['stations'] = data3['stations'].astype(str)

In [11]:
alt.Chart(data3).mark_bar().encode(
    x = alt.X('values:Q'),
    y = alt.Y('hour:O'),
    color = alt.Color('stations:N'),
    tooltip = alt.Tooltip('values')
).properties(width = 650, height = 500, background = '#F9F9F9', padding = 25)

alt.Chart(...)

### Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

In [12]:
data4 = data.copy(deep=False)
data4.drop(['aes', 'tec', 'vde', 'tes', 'ges', 'geas'], axis = 1, inplace = True)

data4_1 = data4.groupby(['month', 'hour'])[['consumption']].mean().reset_index()

In [13]:
data4.loc[data4['month'] >= 9 , 'season'] = "fall"
data4.loc[data4['month'] == 12 , 'season'] = "winter"
data4.loc[data4['month'] <= 8 , 'season'] = "summer"
data4.loc[data4['month'] <= 5 , 'season'] = "spring"
data4.loc[data4['month'] <= 2 , 'season'] = "winter"
data4_2 = data4.groupby(['season', 'hour'])[['consumption']].mean().reset_index()

In [14]:
selection = alt.selection_multi(fields=['month'])
range_ = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4',
          '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#000000']

top1 = alt.Chart().mark_circle(size = 100).encode(
    x = alt.X('hour:N'),
    y = alt.Y('consumption:Q' , scale = alt. Scale(domain = [12000, 23000])),
    color = alt.Color('month:N', scale=alt.Scale(range = range_))
)

top2 = alt.Chart().mark_line().encode(
    x = alt.X('hour:N'),
    y = alt.Y('consumption:Q' , scale = alt. Scale(domain = [12000, 23000])),
    color = alt.Color('month:N', scale=alt.Scale(range = range_)),
)
top = (top1 + top2).properties(width = 850, height = 700).transform_filter(selection)

bottom = alt.Chart().mark_bar().encode(
    x = alt.X('month:O'),
    y = alt.Y('average(consumption):Q'),
    color=alt.condition(selection, alt.value('purple'), alt.value('gray'))
).properties(
    width=600, height=100
).add_selection(
    selection
)

alt.vconcat(
    top, bottom,
    data=data4_1
)

alt.VConcatChart(...)

In [15]:
selection = alt.selection_multi(fields=['season'])
range_ = ['#4363d8', '#f58231', '#3cb44b', '#ffe119']

top1 = alt.Chart().mark_circle(size = 100).encode(
    x = alt.X('hour:N'),
    y = alt.Y('consumption:Q', scale = alt. Scale(domain = [12000, 22100])),
    color = alt.Color('season:N', scale=alt.Scale(range = range_), sort=alt.EncodingSortField(field='consumption', op='mean', order='descending')))

top2 = alt.Chart().mark_line().encode(
    x = alt.X('hour:N'),
    y = alt.Y('consumption:Q', scale = alt. Scale(domain = [12000, 22100])),
    color = alt.Color('season:N', scale=alt.Scale(range = range_), sort=alt.EncodingSortField(field='consumption', op='mean', order='descending')))

top3 = alt.Chart().mark_bar(opacity = 0.4).encode(
    x = alt.X('hour:N'),
    y = alt.Y('consumption:Q', scale = alt. Scale(domain = [12000, 22100]), stack=None),
    color = alt.Color('season:N', scale=alt.Scale(range = range_), sort=alt.EncodingSortField(field='consumption', op='mean', order='descending')))

top = (
    top1 + 
    top2 +
    top3).properties(width = 750, height = 500).transform_filter(selection)

bottom = alt.Chart().mark_bar().encode(
    x = alt.X('season:O', sort=alt.EncodingSortField(field='consumption', op='mean', order='descending')),
    y = alt.Y('average(consumption):Q'),
    color=alt.condition(selection, alt.value('purple'), alt.value('gray'))
).properties(
    width=600, height=100
).add_selection(
    selection
)

alt.vconcat(
    top, bottom,
    data=data4_2
)

alt.VConcatChart(...)

### Як змінюється споживання електроенергії впродовж тижня?

In [16]:
data5 = data.copy(deep=False)
data5.drop(['aes', 'tec', 'vde', 'tes', 'ges', 'geas'], axis = 1, inplace = True)
data5.head()


,consumption,hour,day,month,year
0,16693,24,31,12,2020
1,17805,23,31,12,2020
2,18870,22,31,12,2020
3,19887,21,31,12,2020
4,20387,20,31,12,2020
